In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import freqopttest.glo as glo
import sys

In [ ]:
# sample source 
m = 500
dim = 2
n = m
#ss = data.SSGaussMeanDiff(dim, my=0.2)
ss = data.SSGaussVarDiff(dim)
#ss = data.SSBlobs()
tst_data = ss.sample(m, seed=2)
tr, te = tst_data.split_tr_te(tr_proportion=0.5, seed=10)
#te = tst_data

## smooth CF test

In [ ]:
J = 7
alpha = 0.01
smooth_cf = tst.SmoothCFTest.create_randn(te, J, alpha=alpha, seed=2)
smooth_cf.perform_test(te)

## optimize test frequencies

In [ ]:
op = {'n_test_freqs': J, 'seed': 49, 'max_iter': 300, 
     'batch_proportion': 1.0, 'freqs_step_size': 0.1, 
      'gwidth_step_size': 0.01, 'tol_fun': 1e-4}
# optimize on the training set
test_freqs, gwidth, info = tst.SmoothCFTest.optimize_freqs_width(tr, **op)
scf_opt = tst.SmoothCFTest(test_freqs, gwidth, alpha=alpha)
scf_opt_test = scf_opt.perform_test(te)
scf_opt_test

In [ ]:
# plot optimization results

# trajectories of the Gaussian width
gwidths = info['gwidths']
fig, axs = plt.subplots(2, 2, figsize=(10, 9))
axs[0, 0].plot(gwidths)
axs[0, 0].set_xlabel('iteration')
axs[0, 0].set_ylabel('Gaussian width')
axs[0, 0].set_title('Gaussian width evolution')

# evolution of objective values
objs = info['obj_values']
axs[0, 1].plot(objs)
axs[0, 1].set_title('Objective $\lambda(T)$')

# trajectories of the test locations
# iters x J. X Coordinates of all test locations
locs = info['test_freqs']
for coord in [0, 1]:
    locs_d0 = locs[:, :, coord]
    J = locs_d0.shape[1]
    axs[1, coord].plot(locs_d0)
    axs[1, coord].set_xlabel('iteration')
    axs[1, coord].set_ylabel('index %d of test_locs'%(coord))
    axs[1, coord].set_title('evolution of %d test locations'%J)

print('optimized width: %.3f'%gwidth)

## SCF: optimize just the Gaussian width

In [ ]:
op_gwidth = {'max_iter': 300,'gwidth_step_size': 0.1,  
             'batch_proportion': 1.0, 'tol_fun': 1e-4}
# optimize on the training set
rand_state = np.random.get_state()
np.random.seed(seed=12)
T0_randn = np.random.randn(J, dim)
np.random.set_state(rand_state)

gwidth, info = tst.SmoothCFTest.optimize_gwidth(tr, T0_randn, **op_gwidth)

In [ ]:
# trajectories of the Gaussian width
gwidths = info['gwidths']
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].plot(gwidths)
axs[0].set_xlabel('iteration')
axs[0].set_ylabel('Gaussian width')
axs[0].set_title('Gaussian width evolution')

# evolution of objective values
objs = info['obj_values']
axs[1].plot(objs)
axs[1].set_title('Objective $\lambda(T)$')
